## **Phân tích dữ liệu thông minh - 20KHDL** ##
## **Lab Project - Predict Student Performance from Game Play** ##

---


### ***I. Giới thiệu thành viên và phân công công việc*** ###

|MSSV|Họ tên|Phân công công việc|
|-----|-----|------|
|20127136|Đặng Minh Đức|Quá trình khám phá dữ liệu + Quá trình tiền xử lý dữ liệu|
|20127571|Bùi Văn Nghĩa|b|
|20127676|Nguyễn Vũ Khôi|c|

### ***II. Giới thiệu chung về đồ án*** ###

### ***III. Code*** ###

#### Nạp các thư viện vào cho việc xử lý dữ liệu ####

In [155]:
import pandas as pd
import numpy as np

#### Tải file từ input trong Kaggle ####

In [156]:
#import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
#    for filename in filenames:
#        print(os.path.join(dirname, filename))

#### Quá trình khám phá dữ liệu ####

*1. Số dòng và số cột của train_label và test*

Trước hết, chúng em sẽ đọc hai file ``train_labels.csv`` và ``train.csv`` (cũng chính là hai file chúng em chọn để xử lý). Riêng file ``train.csv``, vì file có kích thước rất lớn nên chúng em đọc kèm theo ``chunksize`` để chia dataset thành từng phần, mỗi phần lấy số dòng là ``chunksize`` (chúng em chọn bằng 10000).

In [157]:
# Đọc file test.csv 
train_labels_df = pd.read_csv("Data/train_labels.csv")#pd.read_csv("/kaggle/input/predict-student-performance-from-game-play/train_labels.csv")
train_labels_df

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1
...,...,...
424111,22100215342220508_q18,1
424112,22100215460321130_q18,1
424113,22100217104993650_q18,1
424114,22100219442786200_q18,1


In [158]:
# Đọc file train.csv 
train_df = pd.read_csv("Data/train.csv", chunksize=2000000)#pd.read_csv("/kaggle/input/predict-student-performance-from-game-play/train.csv", chunksize=20000)
train_df = [train_split_df for train_split_df in train_df]
len(train_df)

14

In [159]:
# Số dòng và số cột của test_df + train_label_df
print('train_label_df có', train_labels_df.shape[0], 'dòng và', train_labels_df.shape[1], 'cột')

num_row_of_train_df = 0
for train_df_chunk in train_df:
    num_row_of_train_df += train_df_chunk.shape[0]
    
print('train_df có', num_row_of_train_df, 'dòng và', train_df[0].shape[1], 'cột')

train_label_df có 424116 dòng và 2 cột
train_df có 26296946 dòng và 20 cột


*2. Ý nghĩa của từng dòng và từng cột*

a. ``train_labels_df``

In [160]:
# Trước hết chúng em sẽ xem tên của từng cột trong train_labels_df
train_labels_col_name = train_labels_df.columns

# In tên của cột trong train_labels_df
print('Các cột trong train_labels_df: ')
for i in range(0, len(train_labels_col_name)):
  print(i + 1, train_labels_col_name[i])

Các cột trong train_labels_df: 
1 session_id
2 correct


In [161]:
# Trích 5 dòng của train_labels_df
train_labels_df.head(5)

,session_id,correct
0,20090312431273200_q1,1
1,20090312433251036_q1,0
2,20090312455206810_q1,1
3,20090313091715820_q1,0
4,20090313571836404_q1,1


**Ý nghĩa của từng dòng trong dữ liệu**: Mỗi dòng của train_labels_df chứa thông tin về độ chính xác trong việc trả lời câu hỏi (với một số thứ tự câu hỏi nào đó) của một phiên. 

**Ý nghĩa của từng cột trong dữ liệu**: 

|Tên cột|Ý nghĩa|
|--------------------|--------------------------------------------------------------------------|
|`session_id`|Mã số ID của một session kèm theo câu hỏi mà học sinh trong session đó đã trả lời|
|`correct`|Kiểm tra độ chính xác trong việc trả lời câu hỏi của học sinh|

b. ``train_df`` \
*Chúng em chỉ xét 1 chunk trong train_df và các chunk khác cũng sẽ có ý nghĩa tương tự*

In [162]:
# Trước hết chúng em sẽ xem tên của từng cột trong train_df
train_col_name = train_df[0].columns

# In tên của cột trong train_labels_df
print('Các cột trong train_df: ')
for i in range(0, len(train_col_name)):
  print(i + 1, train_col_name[i])

Các cột trong train_df: 
1 session_id
2 index
3 elapsed_time
4 event_name
5 name
6 level
7 page
8 room_coor_x
9 room_coor_y
10 screen_coor_x
11 screen_coor_y
12 hover_duration
13 text
14 fqid
15 room_fqid
16 text_fqid
17 fullscreen
18 hq
19 music
20 level_group


In [163]:
# Trích 5 dòng của chunk đầu tiên của train_df
train_df[0].head(5)

,session_id,index,elapsed_time,event_name,name,level,page,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,text,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,undefined,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,"Whatcha doing over there, Jo?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,Just talking to Teddy.,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,NaN,-413.991405,-159.314686,380.0,494.0,NaN,I gotta run to my meeting!,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,NaN,-412.991405,-159.314686,381.0,494.0,NaN,"Can I come, Gramps?",gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4


**Ý nghĩa của từng dòng trong dữ liệu**: Mỗi dòng của train_labels_df là thông tin của mỗi session trong trò chơi. Thông tin cụ thể nhóm em sẽ ghi trong phần **Ý nghĩa của từng cột trong dữ liệu**. \
**Ý nghĩa của từng cột trong dữ liệu**: 

|Tên cột|Ý nghĩa|
|--------------------|--------------------------------------------------------------------------|
|`session_id`|Mã số ID của một session|
|`index`|Số thứ tự của sự kiện cho session đó|
|`elapsed_time`|Thời gian từ lúc bắt đầu session đến lúc sự kiện được ghi nhận|
|`event_name`|Tên loại sự kiện|
|`name`|Tên sự kiện|
|`level`|Cấp độ mà sự kiện đạt đến trong game|
|`page`|Số trang của sự kiện|
|`room_coor_x`|Tọa độ trục hoành mà học sinh ấn chuột trong phòng chơi|
|`room_coor_y`|Tọa độ trục tung mà học sinh ấn chuột trong phòng chơi|
|`screen_coor_x`|Tọa độ trục hoành mà học sinh ấn chuột trên màn hình|
|`screen_coor_y`|Tọa độ trục tung mà học sinh ấn chuột trên màn hình|
|`hover_duration`|Thời gian di chuyển chuột (được tính bằng mi-li-giây)|
|`text`|Nội dung mà người chơi sẽ thấy trong quá trình sự kiện diễn tra trong trò chơi|
|`fqid`|Mã số ID đầy đủ của một sự kiện|
|`room_fqid`|Mã số ID đầy đủ của phòng mà học sinh nhấp chuột vào|
|`text_fqid`|Mã số ID đầy đủ của nội dung mà người chơi sẽ thấy trong quá trình sự kiện diễn tra trong trò chơi|
|`fullscreen`|Kiểm tra xem học sinh có để chế độ toàn màn hình hay không|
|`hq`|Kiểm tra chất lượng của trò chơi có cao hay không|
|`music`|Kiểm tra học sinh có bật nhạc trong quá trình chơi hay không|
|`level_group`|Phạm vi cấp độ - và phạm vi các câu hỏi mà học sinh đạt được|


*3. Kiểu dữ liệu của từng cột*

a. ``train_labels_df``

In [164]:
# In ra kiểu dữ liệu của từng cột của train_labels_df
train_labels_df.dtypes

session_id    object
correct        int64
dtype: object

b. ``train_df``

In [165]:
# In ra kiểu dữ liệu của từng cột của test_df
train_df[0].dtypes

session_id          int64
index               int64
elapsed_time        int64
event_name         object
name               object
level               int64
page              float64
room_coor_x       float64
room_coor_y       float64
screen_coor_x     float64
screen_coor_y     float64
hover_duration    float64
text               object
fqid               object
room_fqid          object
text_fqid          object
fullscreen          int64
hq                  int64
music               int64
level_group        object
dtype: object

**Nhận xét chung cho cả `train_labels_df` lẫn `train_df`**:
- Ở `train_labels_df` có `session_id` có kiểu dữ liệu là `object` và `correct` có kiểu dữ liệu là `int64` (kiểu số).
- Ở `train_df` có `event_name`, `name`, `text`, `fqiq`, `room_fqid`, `text_fqid` và `level_group` đều là kiểu dữ liệu `object`, còn lại là kiểu số (`int64` hay `float64`).
- Trên thực tế, nếu cột nào có kiểu dữ liệu `object` thì các phần tử của cột đó đều có một hoặc nhiều kiểu dữ liệu khác nhau. Để kiểm tra xem cột có kiểu dữ liệu `object` có hợp lý hay không, chúng em sẽ thực hiện code sau.

In [166]:
# Kiểm tra xem cột có kiểu dữ liệu object (session_id) trong train_labels_df có hợp lý hay không
for col_name in train_labels_df.columns:
    col = train_labels_df[col_name]
    if col.dtype == 'object':
      dtype_set = set()
      train_labels_df[col_name].apply(lambda dt: dtype_set.add(type(dt)))
      print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'session_id' có thể có các kiểu dữ liệu: {<class 'str'>}


Từ đây, có thể thấy rằng kiểu dữ liệu `object` đối với cột `session_id` hợp lý vì cột này là mã số ID lẫn câu hỏi nên khi chúng em quan sát dữ liệu trước đó, chúng em nhận ra ngay rằng cột `session_id` phải là dạng chuỗi.

In [167]:
# Kiểm tra xem cột có kiểu dữ liệu object (event_name, name, text, fqiq, room_fqid, text_fqid và level_group) trong train_df có hợp lý hay không
for col_name in train_df[0].columns:
    col = train_df[0][col_name]
    if col.dtype == 'object':
      dtype_set = set()
      train_df[0][col_name].apply(lambda dt: dtype_set.add(type(dt)))
      print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'event_name' có thể có các kiểu dữ liệu: {<class 'str'>}


Cột 'name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text' có thể có các kiểu dữ liệu: {<class 'str'>, <class 'float'>}
Cột 'fqid' có thể có các kiểu dữ liệu: {<class 'str'>, <class 'float'>}
Cột 'room_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text_fqid' có thể có các kiểu dữ liệu: {<class 'str'>, <class 'float'>}
Cột 'level_group' có thể có các kiểu dữ liệu: {<class 'str'>}


Từ đây, có thể thấy rằng kiểu dữ liệu `object` đối với các cột `event_name`, `name`, `room_fqid` và `level_group` hợp lý vì những cột này là mã số ID hoặc tên nên khi chúng em quan sát dữ liệu trước đó, chúng em nhận ra ngay rằng cả bốn cột đều phải là dạng chuỗi. \
Còn kiểu dữ liệu `object` đối với các cột `text`, `fqid` và `text_fqid` không hợp lý vì như chúng em thấy các cột này đều có thể có 2 kiểu dữ liệu. Từ điều này chúng em sẽ tiền xử lý dữ liệu sao cho các cột đều có kiểu dữ liệu hợp lý.

*4. Kiểm tra sự bất thường trong dữ liệu* \
*Đây cũng chính là những cơ sở để nhóm em tiền xử lý dữ liệu.*

a. ``train_labels_df``

a.1. Kiểm tra dữ liệu bị trùng lặp

In [168]:
# Kiểm tra xem có dòng nào bị trùng lặp trong train_labels_df hay không bằng cách sử dụng duplicated và lấy unique, chọn phần tử cuối cùng
train_labels_df.duplicated().unique()[-1]

False

Điều đó chứng tỏ dữ liệu không bị trùng lặp.

a.2. Kiểm tra cột bị thiếu dữ liệu

In [169]:
# Kiểm tra xem có cột nào bị thiếu trong train_labels_df hay không bằng cách sử dụng info
train_labels_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424116 entries, 0 to 424115
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   session_id  424116 non-null  object
 1   correct     424116 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.5+ MB


Sau quá trình quan sát thông tin trên, chúng em thấy rằng không có cột nào thiếu dữ liệu.

b. ``train_df``

b.1. Kiểm tra dữ liệu bị trùng lặp

In [170]:
# Kiểm tra dòng trùng lặp cho từng chunk và tổng hợp lại nguyên train_df có trùng lặp không
check_duplicate_each_chunk = [train_df_chunk.duplicated().unique()[-1] for train_df_chunk in train_df]
if 'True' in check_duplicate_each_chunk:
    print('Điều đó chứng tỏ dữ liệu bị trùng lặp.')
else:
    print('Điều đó chứng tỏ dữ liệu không bị trùng lặp.')

Điều đó chứng tỏ dữ liệu không bị trùng lặp.


b.2. Kiểm tra cột bị thiếu dữ liệu

In [171]:
# Kiểm tra xem có cột nào bị thiếu trong train_df hay không bằng cách sử dụng isnull().sum(), in ra 5 chunk đầu tiên và các chunk khác cũng sẽ tương tự 
chunk_info = [train_df_chunk.isnull().sum() for train_df_chunk in train_df]
chunk_info[0:5]

[session_id              0
 index                   0
 elapsed_time            0
 event_name              0
 name                    0
 level                   0
 page              1957805
 room_coor_x        173061
 room_coor_y        173061
 screen_coor_x      173061
 screen_coor_y      173061
 hover_duration    1832315
 text              1273769
 fqid               621036
 room_fqid               0
 text_fqid         1273762
 fullscreen              0
 hq                      0
 music                   0
 level_group             0
 dtype: int64,
 session_id              0
 index                   0
 elapsed_time            0
 event_name              0
 name                    0
 level                   0
 page              1954106
 room_coor_x        170803
 room_coor_y        170803
 screen_coor_x      170803
 screen_coor_y      170803
 hover_duration    1834463
 text              1287165
 fqid               632767
 room_fqid               0
 text_fqid         1287164
 fullscreen  

Sau quá trình quan sát thông tin trên, nhìn chung, chúng em thấy rằng các cột `page`, `room_coor_x`,`room_coor_y`, `screen_coor_x`, `screen_coor_y`, `hover_duration`, `text`, `fqid` và `text_fqid` thiếu dữ liệu, còn lại không thiếu.

#### Quá trình tiền xử lý dữ liệu ####

a. `train_labels_df`

Dữ liệu ở bảng này đã chuẩn nên không cần phải tiến hành chỉnh sửa mà ta sẽ dùng nó để kết hợp với dữ liệu được tiền xử lí của train_df để tạo bộ X_train cho mô hình

b. `train_df` \
*Đối với dữ liệu này, sau khi mỗi bước tiền xử lý dữ liệu, chúng em chỉ test ở chunk đầu tiên để tránh bị trục trặc trong quá trình chạy code* 

- Trước hết, vì chúng em thấy rằng cột ``page`` và cột ``text`` không mang lại ý nghĩa quan trọng cho việc xử lý nên chúng em sẽ tiến hành xóa hai cột này
- Tiếp đến, chúng em sẽ tiền xử lý với những cột có dữ liệu bị thiếu, bao gồm các cột `room_coor_x`,`room_coor_y`, `screen_coor_x`, `screen_coor_y`, `hover_duration`, `fqid` và `text_fqid` bằng cách điền chúng. Chúng em đề xuất điền dữ liệu bị thiếu đối với các cột đó như sau: \

|Tên cột|Đề xuất|
|--------------------|--------------------------------------------------------------------------|
|`room_coor_x`|Điền tọa độ nhỏ nhất trong cột|
|`room_coor_y`|Điền tọa độ nhỏ nhất trong cột|
|`screen_coor_x`|Điền tọa độ nhỏ nhất trong cột|
|`screen_coor_y`|Điền tọa độ nhỏ nhất trong cột|
|`hover_duration`|Điền là -1 (để phân biệt thời gian xác định (>= 0) và thời gian không xác định)|
|`fqid`|Điền là `undefined_fqid`|
|`text_fqid`|Điền là `undefined_text_fqid`|

- Cuối cùng, đối với các cột có kiểu dữ liệu `object`, trong đó có thể có 2 kiểu dữ liệu như các cột `text`, `fqid` và `text_fqid`, ta sẽ chuyển kiểu dữ liệu `object` thành một kiểu dữ liệu thống nhất. Từ việc quan sát dữ liệu, chúng em nhận ra rằng cả ba cột này đều có dạng là chuỗi vì đó là những cột mang ý nghĩa là *mã ID và nội dung*. Sau đó khi in thông tin bằng hàm info, ta sẽ thấy nó vẫn là object nên chúng em sẽ lấy lại dòng code kiểm tra hợp lý của kiểu dữ liệu object ở phần III.3.b để xem nó đã thành kiểu dữ liệu `str` chưa.

In [172]:
# Thực hiện việc tiền xử lý dữ liệu cho train_df
for i in range(len(train_df)):
    #Xóa cột page và text in ra sau khi xóa
    train_df[i] = train_df[i].drop(columns=['page', 'text'])
    # Điền dữ liệu bị thiếu đối với các cột room_coor_x,room_coor_y, screen_coor_x, screen_coor_y, hover_duration, fqid và text_fqid
    train_df[i] = train_df[i].fillna(value={'room_coor_x': train_df_chunk['room_coor_x'].min(), 'room_coor_y': train_df_chunk['room_coor_y'].min(),
                                      'screen_coor_x': train_df_chunk['screen_coor_x'].min(), 'screen_coor_y': train_df_chunk['screen_coor_y'].min(),
                                      'hover_duration': -1, 'fqid': 'undefined_fqid', 'text_fqid': 'undefined_text_fqid'}) 
    # Chuyển đổi kiểu dữ liệu sang str đối với cột fqid và text_fqid
    train_df[i]['fqid'] = train_df[i]['fqid'].astype('str')
    train_df[i]['text_fqid'] = train_df[i]['text_fqid'].astype('str')

In [173]:
# In chunk đầu tiên của bảng sau khi tiền xử lý
train_df[0]

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,hover_duration,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group
0,20090312431273200,0,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4
1,20090312431273200,1,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
2,20090312431273200,2,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
3,20090312431273200,3,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
4,20090312431273200,4,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,-1.0,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,20110211310637800,472,802324,navigate_click,undefined,9,-0.443521,108.000000,442.0,222.0,-1.0,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12
1999996,20110211310637800,473,802625,navigate_click,undefined,9,-24.435286,114.000000,420.0,216.0,-1.0,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12
1999997,20110211310637800,474,802900,navigate_click,undefined,9,-49.537675,98.000000,396.0,232.0,-1.0,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12
1999998,20110211310637800,475,803451,object_hover,basic,9,-1973.059803,-914.364030,0.0,0.0,525.0,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12


In [174]:
# In thông tin của chunk đầu tiên của train_df
train_df[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 18 columns):
 #   Column          Dtype  
---  ------          -----  
 0   session_id      int64  
 1   index           int64  
 2   elapsed_time    int64  
 3   event_name      object 
 4   name            object 
 5   level           int64  
 6   room_coor_x     float64
 7   room_coor_y     float64
 8   screen_coor_x   float64
 9   screen_coor_y   float64
 10  hover_duration  float64
 11  fqid            object 
 12  room_fqid       object 
 13  text_fqid       object 
 14  fullscreen      int64  
 15  hq              int64  
 16  music           int64  
 17  level_group     object 
dtypes: float64(5), int64(7), object(6)
memory usage: 274.7+ MB


In [175]:
# Kiểm tra xem kiểu dữ liệu của các cột fqid và cột text_fqid được thống nhất là str chưa
for col_name in train_df[0].columns:
  col = train_df[0][col_name]
  if col.dtype == 'object':
    dtype_set = set()
    train_df[0][col_name].apply(lambda dt: dtype_set.add(type(dt)))
    print(f"Cột '{col_name}' có thể có các kiểu dữ liệu: {dtype_set}")

Cột 'event_name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'name' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'room_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'text_fqid' có thể có các kiểu dữ liệu: {<class 'str'>}
Cột 'level_group' có thể có các kiểu dữ liệu: {<class 'str'>}


#### Quá trình tạo đặc trưng ####

Đối với những cột nào có kiểu dữ liệu là string chứa từ 2-4 giá trị riêng biệt, chúng em sẽ tạo đặc trưng bằng cách sử dụng *one-hot encoding*. Để biết cột nào có điều kiện như vậy thì chúng em sẽ chạy code sau:

In [176]:
# In ra mỗi cột chứa bao nhiêu giá trị riêng biệt
for col in train_df[0].columns:
  if train_df[0][col].dtype == 'object':
    print(col, ': ', len(train_df[0][col].unique()))

event_name :  11
name :  6
fqid :  128
room_fqid :  19
text_fqid :  126
level_group :  3


Từ đây, chúng em thấy rằng cần tạo đặc trưng từ cột ``level_group``.

In [177]:
# One-hot encoding đối với cột level_group 
for i in range(len(train_df)):
    level_group_col = train_df[i]['level_group']
    level_group_col = pd.get_dummies(level_group_col, prefix='level_group', dtype=int) # one-hot encoding bằng cách sử dụng hàm get_dummies cho việc đổi từ chuỗi sang 0 hoặc 1 (giá trị số)
    train_df[i] = pd.concat([train_df[i], level_group_col], axis=1)

In [178]:
# In chunk đầu tiên của train_df ra sau khi có các cột one-hot encoding
train_df[0]

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,...,fqid,room_fqid,text_fqid,fullscreen,hq,music,level_group,level_group_0-4,level_group_13-22,level_group_5-12
0,20090312431273200,0,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,intro,tunic.historicalsociety.closet,tunic.historicalsociety.closet.intro,0,0,1,0-4,1,0,0
1,20090312431273200,1,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1,0,0
2,20090312431273200,2,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1,0,0
3,20090312431273200,3,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1,0,0
4,20090312431273200,4,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,...,gramps,tunic.historicalsociety.closet,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1999995,20110211310637800,472,802324,navigate_click,undefined,9,-0.443521,108.000000,442.0,222.0,...,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12,0,0,1
1999996,20110211310637800,473,802625,navigate_click,undefined,9,-24.435286,114.000000,420.0,216.0,...,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12,0,0,1
1999997,20110211310637800,474,802900,navigate_click,undefined,9,-49.537675,98.000000,396.0,232.0,...,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12,0,0,1
1999998,20110211310637800,475,803451,object_hover,basic,9,-1973.059803,-914.364030,0.0,0.0,...,reader,tunic.library.microfiche,undefined_text_fqid,0,0,1,5-12,0,0,1


#### Xây dựng mô hình ####

In [179]:
train_df[0].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000000 entries, 0 to 1999999
Data columns (total 21 columns):
 #   Column             Dtype  
---  ------             -----  
 0   session_id         int64  
 1   index              int64  
 2   elapsed_time       int64  
 3   event_name         object 
 4   name               object 
 5   level              int64  
 6   room_coor_x        float64
 7   room_coor_y        float64
 8   screen_coor_x      float64
 9   screen_coor_y      float64
 10  hover_duration     float64
 11  fqid               object 
 12  room_fqid          object 
 13  text_fqid          object 
 14  fullscreen         int64  
 15  hq                 int64  
 16  music              int64  
 17  level_group        object 
 18  level_group_0-4    int32  
 19  level_group_13-22  int32  
 20  level_group_5-12   int32  
dtypes: float64(5), int32(3), int64(7), object(6)
memory usage: 297.5+ MB


In [180]:
session_id_col = 'session_id'
event_name_col = 'event_name'

In [181]:
train_data = pd.concat(train_df, axis=0)

In [182]:
# do some memory clean-up
import gc

del train_df
gc.collect()

1701

In [183]:
test_data = pd.read_csv('./Data/test.csv')

In [184]:
train_data = pd.concat([train_data, test_data.drop('session_level', axis=1)], axis=0).groupby(['session_id', 'level_group'])

In [185]:
train_data.head()

,session_id,index,elapsed_time,event_name,name,level,room_coor_x,room_coor_y,screen_coor_x,screen_coor_y,...,text_fqid,fullscreen,hq,music,level_group,level_group_0-4,level_group_13-22,level_group_5-12,page,text
0,20090312431273200,0,0,cutscene_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,tunic.historicalsociety.closet.intro,0,0,1,0-4,1.0,0.0,0.0,NaN,NaN
1,20090312431273200,1,1323,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1.0,0.0,0.0,NaN,NaN
2,20090312431273200,2,831,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1.0,0.0,0.0,NaN,NaN
3,20090312431273200,3,1147,person_click,basic,0,-413.991405,-159.314686,380.0,494.0,...,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1.0,0.0,0.0,NaN,NaN
4,20090312431273200,4,1863,person_click,basic,0,-412.991405,-159.314686,381.0,494.0,...,tunic.historicalsociety.closet.gramps.intro_0_...,0,0,1,0-4,1.0,0.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3211,20090312331414616,489,928004,navigate_click,undefined,13,297.729307,-260.559972,656.0,482.0,...,NaN,0,0,1,13-22,NaN,NaN,NaN,NaN,NaN
3212,20090312331414616,490,928836,navigate_click,undefined,13,459.877554,-262.914382,716.0,474.0,...,NaN,0,0,1,13-22,NaN,NaN,NaN,NaN,NaN
3213,20090312331414616,491,929968,navigate_click,undefined,13,622.914322,-354.953570,731.0,531.0,...,NaN,0,0,1,13-22,NaN,NaN,NaN,NaN,NaN
3214,20090312331414616,492,930868,navigate_click,undefined,13,779.643974,-258.638647,791.0,462.0,...,NaN,0,0,1,13-22,NaN,NaN,NaN,NaN,NaN


In [186]:
X = train_data['hq'].unique().index.to_frame(index=False)

In [187]:
X['total_time'] = train_data['elapsed_time'].max().reset_index(drop=True)
X['avg_time'] = ((train_data['elapsed_time'].max() - train_data['elapsed_time'].min()) / (train_data['index'].max() - train_data['index'].min())).reset_index(drop=True)

In [188]:
X[['avg_room_coor_x', 'avg_room_coor_y', 'avg_screen_coor_x', 'avg_screen_coor_y']] = train_data[['room_coor_x', 'room_coor_y', 'screen_coor_x', 'screen_coor_y']].mean().reset_index(drop=True)

In [189]:
X[['n_text', 'n_fqid', 'n_room_fqid', 'n_text_fqid']] = train_data[['text', 'fqid', 'room_fqid', 'text_fqid']].nunique().reset_index(drop=True)

In [190]:
X[['mode_fullscreen', 'mode_hq', 'mode_music']] = train_data[['fullscreen', 'hq', 'music']].apply(pd.DataFrame.mode).reset_index(drop=True)

In [191]:
del train_data
gc.collect()

0

In [192]:
train_labels_df = pd.read_csv('./Data/train_labels.csv')

In [193]:
# create a list with column names q1 through q18
q_cols = []
for i in range(1, 19):
    col = 'q' + str(i)
    q_cols.append(col)

print(q_cols)

['q1', 'q2', 'q3', 'q4', 'q5', 'q6', 'q7', 'q8', 'q9', 'q10', 'q11', 'q12', 'q13', 'q14', 'q15', 'q16', 'q17', 'q18']


In [194]:
q_zeros = pd.DataFrame(np.zeros((len(train_labels_df), 18)), columns=q_cols)

In [195]:
q_train_label = pd.concat([train_labels_df, q_zeros], axis=1)

In [196]:
for col in q_cols:
    q_train_label.loc[(q_train_label['session_id'].str.endswith(col), col)] = q_train_label.loc[(q_train_label['session_id'].str.endswith(col), 'correct')]

In [197]:
# create a function to group question numbers and save in level_group column
def group_qs(session_id):
    q_num = int(session_id.split('_q')[1])
    if 1 <= q_num <= 3:
        return '0-4'
    elif 4 <= q_num <= 13:
        return '5-12'
    else:
        return '13-22'

q_train_label['level_group'] = q_train_label['session_id'].apply(group_qs)

In [198]:
q_train_label['session_id'] = q_train_label['session_id'].apply(lambda x: x.split('_')[0])

In [199]:
X_train_label = q_train_label.drop('correct', axis=1).groupby(['session_id', 'level_group']).sum().reset_index()

In [200]:
X_train_label['session_id'] = X_train_label['session_id'].astype('int64')

In [201]:
X_merged = X.merge(X_train_label, how='left', left_on=['session_id', 'level_group'], right_on=['session_id', 'level_group'])

In [202]:
X_merged = pd.get_dummies(X_merged, columns=['level_group'], dtype='int8')

In [203]:
# split into train and test datasets
test_len = len(test_data.groupby(['session_id', 'level_group'])['index'].min())

X_test = X_merged.iloc[:test_len]
X_train = X_merged.iloc[test_len:]

In [204]:
# split the data into X's and y's
y_train = X_train[q_cols]
X_train.drop(q_cols, axis=1, inplace=True)

X_test.drop(q_cols, axis=1, inplace=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_9516\2632803331.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.drop(q_cols, axis=1, inplace=True)
C:\Users\ACER\AppData\Local\Temp\ipykernel_9516\2632803331.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.drop(q_cols, axis=1, inplace=True)


In [205]:
# Xây dựng và huấn luyện mô hình (ví dụ: Random Forest)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Khởi tạo và huấn luyện mô hình
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm thử
y_pred = model.predict(X_test)

In [206]:
y_preds_df = pd.DataFrame(y_pred, columns=q_cols)

In [207]:
y_preds_df.head()

,q1,q2,q3,q4,q5,q6,q7,q8,q9,q10,q11,q12,q13,q14,q15,q16,q17,q18
0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0
2,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0


In [208]:
output = test_data.groupby(['session_id', 'level_group'])['hq'].unique().index.to_frame(index=False)

In [209]:
output_df = pd.concat([output, y_preds_df], axis=1)

In [210]:
output_df['level_group'] = pd.Categorical(output_df['level_group'], categories=['0-4', '5-12', '13-22'], ordered=True)

In [211]:
output_df = output_df.sort_values(by=['session_id', 'level_group'])

In [212]:
# reset the index and then create a new session level column using index values
output_df.reset_index(drop=True, inplace=True)
output_df['session_level'] = output_df.index

In [213]:
# create lists with session names, correct/incorrect binary labels and session levels
session_ids = []
corrects = []
session_levels = []
for session_id in output_df['session_id'].unique():
    for col in q_cols:
        session_ids.append(str(session_id) + '_' + col)
        q_num = int(col.split('q')[1])
        if 1 <= q_num <= 3:
            corrects.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '0-4')][col]).iloc[0]))
            session_levels.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '0-4')]['session_level']).iloc[0]))
        elif 4 <= q_num <= 13:
            corrects.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '5-12')][col]).iloc[0]))
            session_levels.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '5-12')]['session_level']).iloc[0]))
        else:
            corrects.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '13-22')][col]).iloc[0]))
            session_levels.append(int((output_df[(output_df['session_id'] == session_id) & (output_df['level_group'] == '13-22')]['session_level']).iloc[0]))

In [214]:
# combine the lists into a dataframe and export to CSV
final_output = pd.DataFrame(data=np.array([session_ids, corrects, session_levels]).T, columns=['session_id', 'correct', 'session_level'])

In [215]:
# add session_id numerical and question number column for sorting purposes
final_output['session_id_num'] = final_output['session_id'].apply(lambda x: x.split('_q')[0])
final_output['question_num'] = final_output['session_id'].apply(lambda x: x.split('_q')[1])

In [216]:
# convert dtypes to integer
final_output['session_id_num'] = final_output['session_id_num'].astype('int64')
final_output['question_num'] = final_output['question_num'].astype('int8')

In [217]:
# order the dataframe
final_output = final_output.sort_values(by=['question_num', 'session_id_num']).reset_index(drop=True)

In [218]:
# save the output as a CSV file
final_output.drop(['session_id_num', 'question_num'], axis=1).to_csv('./Data/submission.csv', index=False)

#### Thử nghiệm ####

#### Kết luận ####

#### Hướng phát triển ####